In [58]:
%%writefile telomemore/utils.py
import re
from pathlib import Path
import pysam

class BamFile:
    '''Searches through a folder and stores all bam files in a list.'''
    
    def __init__(self, folder: str):
        self.folder = Path(folder)
        self.files = sorted([bam for bam in self.folder.rglob('*.bam') if bam.is_file()])

class BaseKmerFinder:
    '''Superclass for finding kmer in bam file.'''
    
    def __init__(self, folder: str, pattern: str, out_folder: str):
        self.bam = BamFile(folder)
        self.pattern = re.compile(pattern)
        self.out_folder = Path(out_folder).resolve()
        
    def _number_telomers(self, sequence: str) -> int:
        counts = re.findall(self.pattern, sequence)
        return len(counts)


    
def make_directory(directory: str) -> None:
    '''Create directories which does not yet exists'''
    
    directory = Path(directory)
    if not directory.exists():
        directory.mkdir(exist_ok=True, parents=True)

Overwriting telomemore/utils.py


In [39]:
%%writefile telomemore/countkmers.py
from telomemore.utils import BaseKmerFinder
from typing import Tuple
from collections import Counter, defaultdict
import pysam
from pathlib import Path
import pandas as pd

class CountKmers(BaseKmerFinder):
    '''Finds occurances of kmers in bam file and stores'''
    
    def __init__(self, input_folder: str, pattern: str, output_folder: str):
        super().__init__(input_folder, pattern, output_folder)
        
    def _k_mer_per_read(self, bamfile: Path) -> Tuple[dict, int]:
        '''Counts number of kmer in each read and stores the number in list.'''
        kmer_counts = []
        missed_reads = 0
        for read in bamfile:
            try:
                counts = self._number_telomers(read.seq)
                kmer_counts.append(counts)
            except Exception:
                missed_reads += 1

        counter = Counter(kmer_counts)
        return counter, missed_reads
    
    def run_program(self):
        '''Outputs a csv file with information about kmer occurance for each bam file in the folder.'''
        
        for file in self.bam.files:
            sam = pysam.AlignmentFile(file, 'rb')
            
            print(f'Looking for {self.pattern.pattern} in {file}')
            
            counter, missed_reads = self._k_mer_per_read(sam)
            
            # TODO the folder name instead of file name
            kmer_file = self.out_folder / f'{file.stem}_{self.pattern.pattern}_kmer.csv'
            exception_file = self.out_folder / f'{file.stem}_missed_reads.txt'
            
            number = [key for key in counter.keys()]
            count = [value for value in counter.values()]
            
            pd.DataFrame({'number': number, 'count': count}).to_csv(kmer_file, index=False)
            
            with open(exception_file, 'w+') as missed:
                print(f'Reads that could not be read in {file}: {missed_reads}', file=missed)
            
            print(f'Done with {file}')

Overwriting telomemore/countkmers.py


In [40]:
%%writefile telomemore/counttelomeres.py
from telomemore.utils import BaseKmerFinder
from typing import Tuple
from collections import Counter, defaultdict
import pysam
from pathlib import Path
import pandas as pd

class CountTelomeres(BaseKmerFinder):
    '''Counts number of telomeres in all bam files in the input folder'''

    def __init__(self, input_folder: str, pattern: str, output_folder: str, cutoff: int):
        super().__init__(input_folder, pattern, output_folder)
        self.cutoff = cutoff
        

    def _telomere_counts(self, sam: pysam.AlignmentFile) -> Tuple[dict, dict, int]:
        '''Counts number of telomeres and returns the total reads per cells, telomeres per cells and reads with missed barcodes.'''
        telomeres_cells = defaultdict(int)
        total_reads_cells = defaultdict(int)
        missed_barcodes = 0
    
        for read in sam:
            try:
                total_reads_cells[read.get_tag('CB')] += 1 
                # adds the key to the dict and initializes it to 0 
                telomeres_cells[read.get_tag('CB')]
                if self._number_telomers(read.seq) >= self.cutoff:
                    telomeres_cells[read.get_tag('CB')] += 1
            except Exception:
                missed_barcodes += 1

        return telomeres_cells, total_reads_cells, missed_barcodes
        
    
    def run_program(self) -> None:
        '''Generates teleomere counts for each bam file in the input folder. Writes the results as csv files.'''
        
        for file in self.bam.files:
            
            telomere_file = self.out_folder / f'{file.stem}_telomeres_{self.pattern.pattern}.csv'
            totalreads_file = self.out_folder / f'{file.stem}_total_reads.csv'
            missed_barcods_file = self.out_folder / f'{file.stem}_missed_barcodes.txt'
            
            sam = pysam.AlignmentFile(file, 'rb')
            
            print(f'Looking for {self.pattern.pattern} in {file}')
            
            telomeres_dict, total_dict, missed_barcodes = self._telomere_counts(sam)

            with open(telomere_file, 'a+') as telomere:
                for key, value in telomeres_dict.items():
                    print(f'{key},{value}', file=telomere)

            with open(totalreads_file, 'a+') as total:
                for key, value in total_dict.items():
                    print(f'{key},{value}', file=total)

            with open(missed_barcods_file, 'w+') as missed:
                print(f'Number of missed barcodes = {missed_barcodes}', file=missed)
                
            print(f'Done with {file}')




Overwriting telomemore/counttelomeres.py


In [57]:
%%writefile telomemore/cli.py
import click
from telomemore.countkmers import CountKmers
from telomemore.counttelomeres import CountTelomeres
 
@click.group()
def cli():
    '''WELCOME TO teloMeMore'''
    
    pass

             
@cli.command()
@click.argument('input_folder')
@click.argument('output_folder')
@click.argument('pattern')
def count_kmers(input_folder, output_folder, pattern):
    '''Count the occurances of kmers user specifies and stores the results in a csv file. 
    Finds all bam files in the input folder and output files is stored in output folder.
    
    Required arguments: 
    input_folder,
    output_folder,
    pattern. '''
    
    kmers = CountKmers(input_folder=input_folder, output_folder=output_folder, pattern=pattern)
    kmers.run_program()
    
    
    
@cli.command()
@click.argument('input_folder')
@click.argument('output_folder')
@click.argument('pattern')
@click.argument('cutoff')
def count_telomeres(input_folder, output_folder, pattern, cutoff):
    '''Count the occurances of telomere read in each cell. The telomere is defined by the user, as a read
    which contains the pattern a number of times above the specified cutoff.
    The program goes through all bam files in the input folder and stores the results as csv files in the output
    folder.
    A cell is defined by the cell barcode in the bam file.
    
    Required arguments: 
    input_folder,
    output_folder,
    pattern,
    cutoff.'''
    
    telomeres = CountTelomeres(input_folder=input_folder, output_folder=output_folder, pattern=pattern, cutoff=cutoff)
    telomeres.run_program()

Overwriting telomemore/cli.py


In [ ]:
class WriteCSV:
    pass

class Plot:
    pass



In [44]:
from telomemore.countkmers import CountKmers

test = CountKmers('/Users/williamrosenbaum/Bioinformatics/smallTseq/', 'AAA', 'TESTAR/')

test.run_program()

Looking for AAA in /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/bowtie2/processed/aligned/sample1_umi_tools_extract_cutadapt_bowtie2.bam
Done with /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/bowtie2/processed/aligned/sample1_umi_tools_extract_cutadapt_bowtie2.bam
Looking for AAA in /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/bowtie2/processed/aligned/sample2_umi_tools_extract_cutadapt_bowtie2.bam
Done with /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/bowtie2/processed/aligned/sample2_umi_tools_extract_cutadapt_bowtie2.bam
Looking for AAA in /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/bowtie2/processed/aligned/sample3_umi_tools_extract_cutadapt_bowtie2.bam
Done with /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/bowtie2/processed/aligned/sample3_umi_tools_extract_cutadapt_bowtie2.bam
Looking for AAA in /Users/williamrosenbaum/Bioinformatics/smallTseq/testar_lite/umi_tools/processed/dedup/sam